In [7]:
import seaborn as sns
import numpy as np 
import pandas as pd 
import cv2
import matplotlib.pyplot as plt 
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D
from tensorflow.keras.optimizers import Adam 
# from kerastuner import RandomSearch
import os
import numpy as np
# import torch
import glob
# import torch.nn as nn
# from torchvision.transforms import transforms
# from torch.utils.data import DataLoader
# from torch.optim import Adam
# from torch.autograd import Variable
# import torchvision
import pathlib
import pandas as pd
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
# import torch
# from torch.autograd import Variable
# from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
# from torch.optim import Adam, SGD
import warnings
warnings.filterwarnings("ignore")

In [10]:
train_img = []
train_labels = []

test_img = []
test_labels = []

path_train = (r'C:\Users\Kai Qi Yan\OneDrive\文档\GitHub\Brain_Tumor_Remote\archive\Training')
path_test = (r'C:\Users\Kai Qi Yan\OneDrive\文档\GitHub\Brain_Tumor_Remote\archive\Testing')
img_size= 300
os.listdir(path_train)
# for i in os.listdir(path_train):
#     for j in os.listdir(path_train+i):
#         train_img.append (cv2.resize(cv2.imread(path_train+i+'/'+j), (img_size,img_size))) 
#         train_labels.append(i)
        
# for i in os.listdir(path_test):
#     for j in os.listdir(path_test+i):
#         test_img.append (cv2.resize(cv2.imread(path_test+i+'/'+j), (img_size,img_size))) 
#         test_labels.append(i)
        
# train_img = (np.array(train_img))
# test_img = (np.array(test_img))

['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']